# essentia

Documentation of the Musical Features:

https://essentia.upf.edu/streaming_extractor_music.html

In [1]:
import essentia.standard as es
import glob
import shutil
import pandas as pd
import os
from tqdm.notebook import tqdm
import json
import numpy as np

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
!jupyter nbextension enable --py widgetsnbextension

Traceback (most recent call last):
  File "/home/andreas/.local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/home/andreas/.local/lib/python3.8/site-packages/jupyter_core/application.py", line 254, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/home/andreas/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/home/andreas/.local/lib/python3.8/site-packages/notebook/nbextensions.py", line 980, in start
    super().start()
  File "/home/andreas/.local/lib/python3.8/site-packages/jupyter_core/application.py", line 243, in start
    self.subapp.start()
  File "/home/andreas/.local/lib/python3.8/site-packages/notebook/nbextensions.py", line 888, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/home/andreas/.local/lib/python3.8/site-packages/notebook/nbextensions.py", line 861, in toggle_nbextension_python
    return t

### Compute mean and stdev for all features

Example here:
https://essentia.upf.edu/essentia_python_examples.html

In [3]:
my_dir = os.getcwd()
data  = my_dir + '/wav/'
sel_features = {'Song': []}  # Dic to collect float-like and 1D-array (except rhythm.beats_position) features and put them in a csv
all_features = {}  # Dic to collect all the features disregarding their object type

for file in tqdm(glob.glob(os.path.join(data, '*.wav'))):
    file
    file_name  = os.path.basename(file[0:-4])
    sel_features['Song'].append(file_name)
    all_features[file_name] = {}
    # Compute all features, aggregate only 'mean' and 'stdev' statistics for all low-level, rhythm and tonal frame features
    features, features_frames = es.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                                  rhythmStats=['mean', 'stdev'],
                                                  tonalStats=['mean', 'stdev'])(file)
   
    features_names = sorted(features.descriptorNames())
    for f in features_names:
        all_features[file_name][f] = features[f]
        if type(features[f]) is float:
            if f not in sel_features:
                sel_features[f] = [features[f]]
            else:
                sel_features[f].append(features[f])
        
        elif type(features[f]) is not str:
            if len(features[f].shape) == 1 and f != 'rhythm.beats_position':
                for index, elem in enumerate(features[f]):
                    new_name = f + '_' + str(index)
                    if new_name not in sel_features:
                        sel_features[new_name] = [elem]
                    else:
                        sel_features[new_name].append(elem)

  0%|          | 0/404 [00:00<?, ?it/s]

[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, a

### Save all features as json file

In [4]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """  # SOURCE: https://github.com/mpld3/mpld3/issues/434#issuecomment-340255689
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(all_features, cls=NumpyEncoder)

with open(my_dir + '/RESULTS/essentia_all_features.json', 'w') as f:
    json.dump(dumped, f)

In [5]:
json_file = open(my_dir + '/RESULTS/essentia_all_features.json', 'r')
all_features_dictionary = json.load(json_file)
#print(all_features_dictionary)

### Save selected features as csv file

In [6]:
features_df = pd.DataFrame.from_dict(sel_features)
features_df.to_csv(my_dir + '/RESULTS/essentia.csv', index=False)

In [7]:
sel_features_df = pd.read_csv(my_dir + '/RESULTS/essentia.csv') 
sel_features_df

,Song,lowlevel.average_loudness,lowlevel.barkbands.mean_0,lowlevel.barkbands.mean_1,lowlevel.barkbands.mean_2,lowlevel.barkbands.mean_3,lowlevel.barkbands.mean_4,lowlevel.barkbands.mean_5,lowlevel.barkbands.mean_6,lowlevel.barkbands.mean_7,...,tonal.thpcp_30,tonal.thpcp_31,tonal.thpcp_32,tonal.thpcp_33,tonal.thpcp_34,tonal.thpcp_35,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
0,P_SteelyDan_DoItAga,0.919200,0.000072,0.003222,0.001952,0.001257,0.005099,0.002416,0.001901,0.001734,...,0.280851,0.233988,0.123643,0.099906,0.155215,0.533469,0.629180,0.080176,436.456177,0.709684
1,P_CarlosSantana_Smooth,0.968585,0.000286,0.003258,0.002210,0.002876,0.005484,0.002016,0.001593,0.002648,...,0.626465,0.452395,0.314212,0.303329,0.311549,0.805214,0.570681,0.251540,434.193115,0.958760
2,P_Bilderbuch_Maschin,0.975059,0.002396,0.014416,0.003176,0.004610,0.003895,0.001942,0.001261,0.000746,...,0.287749,0.224426,0.240299,0.242662,0.353981,0.978269,0.641966,0.214921,434.193115,0.914186
3,H_ScroobiusPip_LetEmCo,0.929393,0.000074,0.002706,0.002130,0.003073,0.002656,0.001510,0.001040,0.000787,...,0.571591,0.540440,0.385294,0.300041,0.302483,0.507456,0.570200,0.126817,435.700531,0.830941
4,K_Lalo_Symphon,0.487808,0.000104,0.000633,0.000559,0.000146,0.000550,0.000684,0.000622,0.000725,...,0.368694,0.211757,0.098116,0.114366,0.179965,0.710848,0.697095,0.075291,445.369720,0.707704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,H_Mattafix_BigCity,0.888376,0.002618,0.027940,0.004340,0.001905,0.001861,0.002509,0.000498,0.000677,...,0.185783,0.193509,0.249541,0.276839,0.437165,0.851485,0.699806,0.155335,434.193115,0.883734
400,H_Deli_5_32,0.777475,0.003085,0.030101,0.003964,0.001658,0.002758,0.001906,0.001432,0.000425,...,0.357339,0.190742,0.147348,0.322075,0.314995,0.863443,0.466825,0.367873,434.193115,0.994924
401,K_Casella_Italia,0.757737,0.000016,0.004551,0.003671,0.001748,0.004404,0.003242,0.001783,0.001062,...,0.735235,0.501088,0.179014,0.100353,0.164087,0.647657,0.600507,0.012608,442.804626,0.693119
402,H_KRS-One_SoundOf,0.966410,0.000578,0.013758,0.006011,0.003454,0.001346,0.001270,0.000548,0.000499,...,0.635541,0.633552,0.681285,0.660742,0.516855,0.727529,0.340409,0.289884,434.193115,0.904275


**NOTE:** I have ignored rhythm.beats_position in the csv because the lenght of arrays varies across songs and makes problems when fitting them in a table. Categorical variables are also ignored in the csv.